# Prepare Workloads

In this notebook, we will try to prepare the workloads that we will be applying
to the deployments. Potentially, this leads to functions that will be called
when we want to send traffic to the deployment and we want to measure the system's
performance.

In [1]:
%load_ext autoreload
%autoreload 2

from tqdm.auto import tqdm
import itertools

import os
import time
import traceback

# my imports
from helpers import kube
from helpers import workload
from helpers import util
from helpers import request_funcs

fetching imagenet v2
resizing images


  0%|          | 0/100 [00:00<?, ?it/s]

converting to bentoml files


  0%|          | 0/100 [00:00<?, ?it/s]

extracting base64 files


  0%|          | 0/100 [00:00<?, ?it/s]

preprocessing for mobilenet


  0%|          | 0/100 [00:00<?, ?it/s]

In [2]:
config_names = [
    'bentoml-iris-250m-512mb',
    'bentoml-onnx-resnet50-250m-512mb',
    'tfserving-resnetv2-250m-512mb',
    'tfserving-mobilenetv1-250m-512mb',
]

workload_configs = {}
for exp_config_name in config_names:
    exp_file = f"deployments/{exp_config_name}.json"
    workload_spec = util.load_json_file(exp_file)
    workload_configs[workload_spec['name']] = workload_spec


In [3]:
def experiment_batch(warmup_req_count, total_req_count, batch_size, service_name, cpu_m):
    # apply new cpu configuration to the deployment
    ram_mb = int(cpu_m * 1.5)
    request_limit_override = f"'cpu={cpu_m}m,memory={ram_mb}Mi'"
    print('Request Limit Overrid:', request_limit_override)

    workload_spec = workload_configs[service_name]
    # override request and limit values
    workload_spec['opts']['--request'] = request_limit_override
    workload_spec['opts']['--limit'] = request_limit_override
    kn_command = kube.get_kn_command(**workload_spec)
    print(kn_command)
    !{kn_command}
    print('waiting for settings to converge')
    time.sleep(10)

    # an array as the same length as 
    batch_results = {
        'response_times_ms': [],
    }
    reqs_failed = 0
    reqs_succeeded = 0

    print('warming up...')
    for _ in range(warmup_req_count):
        try:
            # discard the results
            request_funcs.workload_funcs[service_name](batch_size=batch_size)
        except Exception:
            print('exception occured:')
            traceback.print_exc()

    # running the main workload
    print(f'running {service_name} workload function, batch_size: {batch_size}')
    for _ in tqdm(range(total_req_count)):
        try:
            result = request_funcs.workload_funcs[service_name](batch_size=batch_size)
            batch_results['response_times_ms'].append(result['response_time_ms'])
            reqs_succeeded += 1
        except Exception:
            print('exception occured:')
            traceback.print_exc()
            reqs_failed += 1

    batch_results.update({
        'reqs_failed': reqs_failed,
        'reqs_succeeded': reqs_succeeded,
        'ram_mb': ram_mb,
    })

    return batch_results

def perform_experiment_batch(config):
    # configurables = ['batch_size']
    configurables = [k for k in config if isinstance(config[k], list)]
    print(f'configurables: {configurables}')

    # make a copy of config
    config_base = {k:config[k] for k in config if k not in configurables}
    configurable_base = {k:config[k] for k in config if k in configurables}

    results = []

    config_combinations_keys = list(configurable_base.keys())
    for config_combination in itertools.product(*configurable_base.values()):
        configurable = {config_combinations_keys[i]: config_combination[i]  for i in range(len(config_combinations_keys))}
        
        print(configurable)
        new_config = {**config_base}
        new_config.update(configurable)

        result = experiment_batch(**new_config)
        new_config.update(result)
        results.append(new_config)
        
    return results

config = {
    'warmup_req_count': 20,
    'total_req_count': 10,
    'service_name': 'bentoml-iris',
    'batch_size': [1,10,50,100],
    # 'cpu_m': [250,500,1000,1500,2000],
    'cpu_m': [250,1000],
}
results = perform_experiment_batch(config)

configurables: ['batch_size', 'cpu_m']
{'batch_size': 1, 'cpu_m': 250}
Request Limit Overrid: 'cpu=250m,memory=375Mi'
kn service apply bentoml-iris --image ghcr.io/nimamahmoudi/bentoml-iris-classifier:20210429201447 \
  --limit 'cpu=250m,memory=375Mi' \
  --request 'cpu=250m,memory=375Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
Applying service 'bentoml-iris' in namespace 'default':

  0.337s The Configuration is still working to reflect the latest desired specification.
 11.597s Traffic is not yet migrated to the latest revision.
 11.653s Ingress has not yet been reconciled.
 11.740s Waiting for load balancer to be ready
 11.921s Ready to serve.

Service 'bentoml-iris' applied to latest revision 'bentoml-iris-00102' is available at URL:
http://bentoml-iris.default.kn.nima-dev.com
waiting for settings to converge
warming up...
running bentoml-iris workload function, batch_size: 1


  0%|          | 0/10 [00:00<?, ?it/s]

{'batch_size': 1, 'cpu_m': 1000}
Request Limit Overrid: 'cpu=1000m,memory=1500Mi'
kn service apply bentoml-iris --image ghcr.io/nimamahmoudi/bentoml-iris-classifier:20210429201447 \
  --limit 'cpu=1000m,memory=1500Mi' \
  --request 'cpu=1000m,memory=1500Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
Applying service 'bentoml-iris' in namespace 'default':

  0.087s The Configuration is still working to reflect the latest desired specification.
  3.661s Traffic is not yet migrated to the latest revision.
  3.704s Ingress has not yet been reconciled.
  3.738s Waiting for load balancer to be ready
  3.957s Ready to serve.

Service 'bentoml-iris' applied to latest revision 'bentoml-iris-00103' is available at URL:
http://bentoml-iris.default.kn.nima-dev.com
waiting for settings to converge
warming up...
running bentoml-iris workload function, batch_size: 1


  0%|          | 0/10 [00:00<?, ?it/s]

{'batch_size': 10, 'cpu_m': 250}
Request Limit Overrid: 'cpu=250m,memory=375Mi'
kn service apply bentoml-iris --image ghcr.io/nimamahmoudi/bentoml-iris-classifier:20210429201447 \
  --limit 'cpu=250m,memory=375Mi' \
  --request 'cpu=250m,memory=375Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
Applying service 'bentoml-iris' in namespace 'default':

  0.046s The Configuration is still working to reflect the latest desired specification.
 12.437s Traffic is not yet migrated to the latest revision.
 12.506s Ingress has not yet been reconciled.
 12.534s Waiting for load balancer to be ready
 12.716s Ready to serve.

Service 'bentoml-iris' applied to latest revision 'bentoml-iris-00104' is available at URL:
http://bentoml-iris.default.kn.nima-dev.com
waiting for settings to converge
warming up...
running bentoml-iris workload function, batch_size: 10


...

  0%|          | 0/10 [00:00<?, ?it/s]

{'batch_size': 100, 'cpu_m': 250}
Request Limit Overrid: 'cpu=250m,memory=375Mi'
kn service apply bentoml-iris --image ghcr.io/nimamahmoudi/bentoml-iris-classifier:20210429201447 \
  --limit 'cpu=250m,memory=375Mi' \
  --request 'cpu=250m,memory=375Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
Applying service 'bentoml-iris' in namespace 'default':

  0.053s The Configuration is still working to reflect the latest desired specification.
  7.870s Traffic is not yet migrated to the latest revision.
  7.900s Ingress has not yet been reconciled.
  8.036s Waiting for load balancer to be ready
  8.169s Ready to serve.

Service 'bentoml-iris' applied to latest revision 'bentoml-iris-00108' is available at URL:
http://bentoml-iris.default.kn.nima-dev.com
waiting for settings to converge
warming up...
running bentoml-iris workload function, batch_size: 100


  0%|          | 0/10 [00:00<?, ?it/s]

{'batch_size': 100, 'cpu_m': 1000}
Request Limit Overrid: 'cpu=1000m,memory=1500Mi'
kn service apply bentoml-iris --image ghcr.io/nimamahmoudi/bentoml-iris-classifier:20210429201447 \
  --limit 'cpu=1000m,memory=1500Mi' \
  --request 'cpu=1000m,memory=1500Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
Applying service 'bentoml-iris' in namespace 'default':

  0.044s The Configuration is still working to reflect the latest desired specification.
  5.365s Traffic is not yet migrated to the latest revision.
  5.461s Ingress has not yet been reconciled.
  5.511s Waiting for load balancer to be ready
  5.832s Ready to serve.

Service 'bentoml-iris' applied to latest revision 'bentoml-iris-00109' is available at URL:
http://bentoml-iris.default.kn.nima-dev.com
waiting for settings to converge
warming up...
running bentoml-iris workload function, batch_size: 100


  0%|          | 0/10 [00:00<?, ?it/s]

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def post_process(df):
    df['resp_time_avg'] = df.apply(lambda x: np.mean(x['response_times_ms']), axis=1)
    for percentile in [50,90,95,99]:
        df[f'resp_time_p{percentile}'] = df.apply(lambda x: np.percentile(x['response_times_ms'], percentile), axis=1)
    return df


df = pd.DataFrame(data=results)
# df = post_process(df)
df

,warmup_req_count,total_req_count,service_name,batch_size,cpu_m,response_times_ms,reqs_failed,reqs_succeeded,ram_mb
0,20,10,bentoml-iris,1,250,"[22.009, 40.699, 19.105, 75.583, 23.679, 73.61...",0,10,375
1,20,10,bentoml-iris,1,1000,"[19.64, 16.182, 16.684, 20.708000000000002, 16...",0,10,1500
2,20,10,bentoml-iris,10,250,"[53.98, 19.314, 78.347, 86.346, 19.885, 17.665...",0,10,375
3,20,10,bentoml-iris,10,1000,"[29.571, 26.528, 26.236, 25.723, 22.947, 23.30...",0,10,1500
4,20,10,bentoml-iris,50,250,"[35.887, 41.457, 35.39, 55.244, 88.897, 38.357...",0,10,375
5,20,10,bentoml-iris,50,1000,"[21.679000000000002, 39.723, 23.309, 19.949, 2...",0,10,1500
6,20,10,bentoml-iris,100,250,"[38.538000000000004, 38.900999999999996, 84.63...",0,10,375
7,20,10,bentoml-iris,100,1000,"[41.816, 53.155, 42.742000000000004, 456.281, ...",0,10,1500


In [5]:
all_configs = [
    {
        'warmup_req_count': 200,
        'total_req_count': 1000,
        'service_name': 'bentoml-iris',
        'batch_size': [1,5,10,20,50,100],
        'cpu_m': [250,500,1000,1500,2000],
    },
    {
        'warmup_req_count': 20,
        'total_req_count': 1000,
        'service_name': 'tfserving-resnetv2',
        'batch_size': [1,2,3,5,10],
        'cpu_m': [250,500,1000,1500,2000],
    },
    {
        'warmup_req_count': 20,
        'total_req_count': 1000,
        'service_name': 'tfserving-mobilenetv1',
        'batch_size': [1,2,3,5],
        'cpu_m': [250,500,1000,1500,2000],
    },
    {
        'warmup_req_count': 20,
        'total_req_count': 1000,
        'service_name': 'bentoml-onnx-resnet50',
        'batch_size': [1,2,3,5,10],
        'cpu_m': [250,500,1000,1500,2000],
    },
]

# results_folder = './results/batch_experiments_default'
results_folder = './results/batch_experiments_var_cpu'

!mkdir -p {results_folder}
for config in all_configs:
    service_name = config['service_name']

    results = perform_experiment_batch(config)
    df = pd.DataFrame(data=results)

    now = util.get_time_with_tz()
    res_name = now.strftime(f'{service_name}_%Y-%m-%d_%H-%M-%S')
    print('res_name:', res_name)
    df.to_csv(f'{results_folder}/{res_name}.csv')

configurables: ['batch_size', 'cpu_m']
{'batch_size': 1, 'cpu_m': 250}
Request Limit Overrid: 'cpu=250m,memory=375Mi'
kn service apply bentoml-iris --image ghcr.io/nimamahmoudi/bentoml-iris-classifier:20210429201447 \
  --limit 'cpu=250m,memory=375Mi' \
  --request 'cpu=250m,memory=375Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
Applying service 'bentoml-iris' in namespace 'default':

  7.130s Traffic is not yet migrated to the latest revision.
  7.193s Ingress has not yet been reconciled.
  7.277s Waiting for load balancer to be ready
  7.819s Ready to serve.

Service 'bentoml-iris' applied to latest revision 'bentoml-iris-00110' is available at URL:
http://bentoml-iris.default.kn.nima-dev.com
waiting for settings to converge
warming up...
running bentoml-iris workload function, batch_size: 1


  0%|          | 0/1000 [00:00<?, ?it/s]

{'batch_size': 1, 'cpu_m': 500}
Request Limit Overrid: 'cpu=500m,memory=750Mi'
kn service apply bentoml-iris --image ghcr.io/nimamahmoudi/bentoml-iris-classifier:20210429201447 \
  --limit 'cpu=500m,memory=750Mi' \
  --request 'cpu=500m,memory=750Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
Applying service 'bentoml-iris' in namespace 'default':

  0.052s The Configuration is still working to reflect the latest desired specification.
  5.596s Traffic is not yet migrated to the latest revision.
  5.640s Ingress has not yet been reconciled.
  5.738s Waiting for load balancer to be ready
  5.897s Ready to serve.

Service 'bentoml-iris' applied to latest revision 'bentoml-iris-00111' is available at URL:
http://bentoml-iris.default.kn.nima-dev.com
waiting for settings to converge
warming up...
running bentoml-iris workload function, batch_size: 1


  0%|          | 0/1000 [00:00<?, ?it/s]

{'batch_size': 1, 'cpu_m': 1000}
Request Limit Overrid: 'cpu=1000m,memory=1500Mi'
kn service apply bentoml-iris --image ghcr.io/nimamahmoudi/bentoml-iris-classifier:20210429201447 \
  --limit 'cpu=1000m,memory=1500Mi' \
  --request 'cpu=1000m,memory=1500Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
Applying service 'bentoml-iris' in namespace 'default':

  0.084s The Configuration is still working to reflect the latest desired specification.
 12.263s Traffic is not yet migrated to the latest revision.
 12.458s Ingress has not yet been reconciled.
 12.487s Waiting for load balancer to be ready
 12.643s Ready to serve.

Service 'bentoml-iris' applied to latest revision 'bentoml-iris-00112' is available at URL:
http://bentoml-iris.default.kn.nima-dev.com
waiting for settings to converge
warming up...
running bentoml-iris workload function, batch_size: 1


...

{'batch_size': 10, 'cpu_m': 1500}
Request Limit Overrid: 'cpu=1500m,memory=2250Mi'
kn service apply bentoml-onnx-resnet50 --image ghcr.io/nimamahmoudi/bentoml-onnx-resnet50-b64:20210505124300 \
  --limit 'cpu=1500m,memory=2250Mi' \
  --request 'cpu=1500m,memory=2250Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
Applying service 'bentoml-onnx-resnet50' in namespace 'default':

  0.070s The Configuration is still working to reflect the latest desired specification.
 12.519s Traffic is not yet migrated to the latest revision.
 12.557s Ingress has not yet been reconciled.
 12.697s Waiting for load balancer to be ready
 13.222s Ready to serve.

Service 'bentoml-onnx-resnet50' applied to latest revision 'bentoml-onnx-resnet50-00029' is available at URL:
http://bentoml-onnx-resnet50.default.kn.nima-dev.com
waiting for settings to converge
warming up...
running bentoml-onnx-resnet50 workload function, batch_size: 10


  0%|          | 0/1000 [00:00<?, ?it/s]

{'batch_size': 10, 'cpu_m': 2000}
Request Limit Overrid: 'cpu=2000m,memory=3000Mi'
kn service apply bentoml-onnx-resnet50 --image ghcr.io/nimamahmoudi/bentoml-onnx-resnet50-b64:20210505124300 \
  --limit 'cpu=2000m,memory=3000Mi' \
  --request 'cpu=2000m,memory=3000Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
Applying service 'bentoml-onnx-resnet50' in namespace 'default':

  0.083s The Configuration is still working to reflect the latest desired specification.
  4.540s Traffic is not yet migrated to the latest revision.
  4.604s Ingress has not yet been reconciled.
  4.711s Waiting for load balancer to be ready
  4.852s Ready to serve.

Service 'bentoml-onnx-resnet50' applied to latest revision 'bentoml-onnx-resnet50-00030' is available at URL:
http://bentoml-onnx-resnet50.default.kn.nima-dev.com
waiting for settings to converge
warming up...
running bentoml-onnx-resnet50 workload function, batch_size: 10


  0%|          | 0/1000 [00:00<?, ?it/s]

res_name: bentoml-onnx-resnet50_2021-05-10_05-14-13
